In [1]:
import mysql.connector

#### 1. Create Connection

In [3]:
mydb = mysql.connector.connect(
    host='192.168.35.159', 
    port=3306,
    user='newrandom', 
    password=input()
    )

#### Create Connection 예제 1
- Local Database 연결

In [5]:
local = mysql.connector.connect(
    host='localhost',
    user = 'root',
    password=input()

)

#### Create Connection 예제2
- AWS RDS(database -1) 연결

In [6]:
remote = mysql.connector.connect(
    host = 'END POINT',
    port = 3306,
    user = 'root',
    password = 'password'       # AWS 접속 시 해당 문구들 변경할 것.
)

InterfaceError: 2003: Can't connect to MySQL server on '%-.100s:%u' (%s) (Warning: %u format: a real number is required, not str)

#### Close Database


In [7]:
local.close()

In [9]:
mydb.close()

#### Create Connection 예제 1
- Local MySQL 의 zerobase 에 연결

In [10]:
local = mysql.connector.connect(
    host = 'localhost',
    user = 'root',
    password = input(),
    database = 'zerobase'       # zerobase database가 있을 때 가능
)

ProgrammingError: 1049 (42000): Unknown database 'zerobase'

In [11]:
local.close()

- AWS RDS(database-1)의 zerobase 에 연결

In [12]:
remote = mysql.connector.connect(
    host = 'ENDPOINT',
    port=  3306,
    user = 'admin', 
    password = 'zerobase',
    database = 'zerobase'
)

InterfaceError: 2003: Can't connect to MySQL server on '%-.100s:%u' (%s) (Warning: %u format: a real number is required, not str)

In [13]:
remote.close()

NameError: name 'remote' is not defined

# Execute SQL
쿼리를 실행하기 위한 코드

In [1]:
import mysql.connector

#### Execute SQL 예제1
- table 생성

In [4]:
remote = mysql.connector.connect(
    host='192.168.35.159',
    port = 3306,
    user = 'newrandom',
    password = input(),
    database = 'zerobase'
)

cur = remote.cursor()
cur.execute('CREATE TABLE sql_file (ID int, filename varchar(16))')

remote.close()

#### Execute SQL 예제2
- 테이블 삭제

In [5]:
remote = mysql.connector.connect(
    host='192.168.35.159',
    port = 3306,
    user = 'newrandom',
    password = input(),
    database = 'zerobase'
)

cur = remote.cursor()
cur.execute('DROP TABLE sql_file')

remote.close()

#### Execute SQL File 예제 1
- test03.sql파일 생성

- test03.sql 파일 실행

In [2]:
remote = mysql.connector.connect(
    host='192.168.35.159',
    port = 3306,
    user = 'newrandom',
    password = input(),
    database = 'zerobase'
)

cur = remote.cursor()
sql = open('./test03.sql').read()      # 실행할 sql 파일의 경로
cur.execute(sql)

remote.close()

#### Execute SQL File 예제 2
- SQL file 내에 Query가 여러개 존재하는 경우

- test04.sql 생성

- 실행

In [11]:
remote = mysql.connector.connect(
    host='192.168.35.159',
    port = 3306,
    user = 'newrandom',
    password = input(),
    database = 'zerobase'
)

cur = remote.cursor()
sql = open('./test04.sql').read()
# cur.execute(sql)      ## 다중쿼리는 일반적인 execute 로 할 수 없음


## 다중 쿼리인 경우에는 multi=True 옵션을 넣어주어야 한다.
for result_iterator in cur.execute(sql, multi=True):
    if result_iterator.with_rows:               ## boolean 형태, row면 True, row가 아니면 False
        print(result_iterator.fetchall())       ## 실행된 결과를 한줄씩 보여줌.
    else:
        print(result_iterator.statement)        ## statement ?

remote.commit()
remote.close()

## sql > python > pandas


In [41]:
remote = mysql.connector.connect(
    host = '192.168.35.159',
    port = 3306,
    user = 'newrandom',
    password = input(),
    database = 'zerobase'
)

cur = remote.cursor(buffered=True)
cur.execute('select * from sql_file')
result = cur.fetchall()

remote.close()

In [44]:
import pandas as pd
df = pd.DataFrame(result)
df

,0,1
0,1,test01.sql
1,2,test02.sql
2,3,test03.sql
3,4,test04.sql


# Read CSV

- police_station.csv 파일 pandas로 읽어와 데이터 확인


In [45]:
import pandas as pd
df = pd.read_csv('../csv/police_station.csv', )
df.head()

,서울특별시경찰청,서울시 종로구 사직로8길 31
0,서울중부경찰서,서울특별시 중구 수표로 27
1,서울종로경찰서,서울특별시 종로구 율곡로 46
2,서울남대문경찰서,서울특별시 중구 한강대로 410
3,서울서대문경찰서,서울특별시 서대문구 통일로 113
4,서울혜화경찰서,서울특별시 종로구 창경궁로 112-16


#### CSV to SQL

In [54]:
import mysql.connector
import pandas as pd

remote = mysql.connector.connect(
    host='192.168.35.159',
    port = 3306,
    user = 'newrandom',
    password=input(),
    database='zerobase'
)

cur = remote.cursor(buffered=True)

# ===============DataFrame load================== #
df = pd.read_csv('../csv/police_station.csv', header=None)

query = 'INSERT INTO police_station (name, address) VALUES (%s,%s)'

for i , row in df.iterrows():
    cur.execute(query, tuple(row))
    print(tuple(row))
    remote.commit()

# cur.execute('SELECT * FROM police_station')

# result = cur.fetchall()
# for row in result:
#     print(row)


remote.close()

('서울특별시경찰청', '서울시 종로구 사직로8길 31')
('서울중부경찰서', '서울특별시 중구 수표로 27')
('서울종로경찰서', '서울특별시 종로구 율곡로 46')
('서울남대문경찰서', '서울특별시 중구 한강대로 410')
('서울서대문경찰서', '서울특별시 서대문구 통일로 113')
('서울혜화경찰서', '서울특별시 종로구 창경궁로 112-16')
('서울용산경찰서', '서울특별시 용산구 원효로89길 24')
('서울성북경찰서', '서울특별시 성북구 보문로 170')
('서울동대문경찰서', '서울특별시 동대문구 약령시로 21길 29')
('서울마포경찰서', '서울특별시 마포구 마포대로 183')
('서울영등포경찰서', '서울특별시 영등포구 국회대로 608')
('서울성동경찰서', '서울특별시 성동구 왕십리광장로 9')
('서울동작경찰서', '서울특별시 동작구 노량진로 148')
('서울광진경찰서', '서울특별시 광진구 광나루로 447 광진소방서 임시청사 (능동)')
('서울서부경찰서', '서울특별시 은평구 은평로9길 15')
('서울강북경찰서', '서울특별시 강북구 오패산로 406')
('서울금천경찰서', '서울특별시 관악구 남부순환로 1435')
('서울중랑경찰서', '서울특별시 중랑구 중랑역로 137')
('서울강남경찰서', '서울특별시 강남구 테헤란로 114길 11')
('서울관악경찰서', '서울특별시 관악구 관악로5길 33')
('서울강서경찰서', '서울특별시 양천구 화곡로 73')
('서울강동경찰서', '서울특별시 강동구 성내로 33')
('서울종암경찰서', '서울특별시 성북구 종암로 135')
('서울구로경찰서', '서울특별시 구로구 가마산로 235')
('서울서초경찰서', '서울특별시 서초구 반포대로 179')
('서울양천경찰서', '서울특별시 양천구 목동동로 99')
('서울송파경찰서', '서울특별시 송파구 중대로 221')
('서울노원경찰서', '서울특별시 노원구 노원로 283')
('서울방배경찰서', '서울특별시 서초구 방배천로 5

In [58]:
import mysql.connector
import pandas as pd

remote = mysql.connector.connect(
    host='192.168.35.159',
    port = 3306,
    user = 'newrandom',
    password=input(),
    database='zerobase'
)

cur = remote.cursor(buffered=True)

df = pd.read_csv('../csv/2020_crime.csv', encoding='euc-kr')

query = 'INSERT INTO crime_status (year, police_station, crime_type, status_type, case_number) VALUES ("2020", %s, %s, %s, %s)'

for i, row in df.iterrows():
    cur.execute(query, tuple(row))
    print(tuple(row))
    remote.commit()

print('============================')
cur.execute('SELECT * FROM crime_status')

result = cur.fetchall()
for row in result:
    print(row)

print('======================')
df_ = pd.DataFrame(result)
df_.head()

remote.close()

('중부', '살인', '발생', 1)
('중부', '살인', '검거', 1)
('중부', '강도', '발생', 3)
('중부', '강도', '검거', 4)
('중부', '강간,추행', '발생', 113)
('중부', '강간,추행', '검거', 76)
('중부', '절도', '발생', 943)
('중부', '절도', '검거', 495)
('중부', '폭력', '발생', 997)
('중부', '폭력', '검거', 830)
('종로', '살인', '발생', 9)
('종로', '살인', '검거', 6)
('종로', '강도', '발생', 4)
('종로', '강도', '검거', 4)
('종로', '강간,추행', '발생', 115)
('종로', '강간,추행', '검거', 93)
('종로', '절도', '발생', 602)
('종로', '절도', '검거', 284)
('종로', '폭력', '발생', 964)
('종로', '폭력', '검거', 792)
('남대문', '살인', '발생', 1)
('남대문', '살인', '검거', 1)
('남대문', '강도', '발생', 2)
('남대문', '강도', '검거', 2)
('남대문', '강간,추행', '발생', 73)
('남대문', '강간,추행', '검거', 52)
('남대문', '절도', '발생', 699)
('남대문', '절도', '검거', 368)
('남대문', '폭력', '발생', 579)
('남대문', '폭력', '검거', 494)
('서대문', '살인', '발생', 3)
('서대문', '살인', '검거', 3)
('서대문', '강도', '발생', 2)
('서대문', '강도', '검거', 1)
('서대문', '강간,추행', '발생', 159)
('서대문', '강간,추행', '검거', 149)
('서대문', '절도', '발생', 1077)
('서대문', '절도', '검거', 524)
('서대문', '폭력', '발생', 1292)
('서대문', '폭력', '검거', 1029)
('혜화', '살인', '발생', 0)
('혜화', 

,0,1,2,3,4
0,2020,중부,살인,발생,1
1,2020,중부,살인,검거,1
2,2020,중부,강도,발생,3
3,2020,중부,강도,검거,4
4,2020,중부,"강간,추행",발생,113
